# Mixed-Integer Programming (MIP) Demo - Tourist Trip Optimizer

This notebook demonstrates the **Mixed-Integer Programming (MIP)** approach to solving the Tourist Trip Design Problem (TTDP).

## What You'll Learn:
- How to use the MIP solver with real Sri Lankan POI data
- How to customize trip requirements and find optimal solutions
- How MIP guarantees mathematical optimality
- When to choose MIP over heuristic approaches

## Key Advantages of MIP:
- 🎯 **Optimal**: Guaranteed mathematically optimal solution
- ✅ **Exact**: No approximations or heuristics
- 📊 **Provable**: Solution quality can be proven

## Trade-offs:
- ⏱️ **Slower**: Can take minutes to hours for large problems
- 📏 **Limited Scale**: Best for smaller problem instances (< 100 POIs)

---

## Setup: Import Libraries and Load Data

In [ ]:
# Import required libraries
import sys
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

# Import our custom modules
sys.path.append('../src')
from mip_solver import MIPSolverTTDP

print("✓ Libraries imported successfully")

In [ ]:
# Load preprocessed Sri Lankan POI data
pois_full = pd.read_csv('../data/processed/pois_processed.csv')
travel_matrix_full = np.load('../data/processed/travel_time_matrix.npy')

print(f"Loaded {len(pois_full)} Points of Interest")
print(f"\nAvailable categories:")
print(pois_full['category'].value_counts())

---
## Use Case 1: Premium Cultural Tour (2 Days)

**Scenario**: A traveler wants the **absolute best** cultural itinerary, willing to wait for the optimal solution. Focus on top-rated cultural sites for a premium 2-day experience.

**Requirements**:
- Top 30 cultural POIs only (to keep solve time reasonable)
- 2 days trip
- 8 hours per day
- Guaranteed optimal solution

In [ ]:
# Filter and select top cultural POIs
cultural_pois = pois_full[
    (pois_full['category'] == 'Cultural') & 
    (pois_full['rating'] >= 4.0)
].nlargest(30, 'interest_score').reset_index(drop=True)

# Get corresponding travel time matrix
cultural_indices = pois_full[
    (pois_full['category'] == 'Cultural') & 
    (pois_full['rating'] >= 4.0)
].nlargest(30, 'interest_score').index.tolist()
cultural_travel_matrix = travel_matrix_full[np.ix_(cultural_indices, cultural_indices)]

print(f"Selected top {len(cultural_pois)} cultural POIs")
print(f"\nTop 10 attractions:")
print(cultural_pois.head(10)[['name', 'rating', 'interest_score', 'visit_duration']])

In [ ]:
# Configure MIP solver
mip_cultural = MIPSolverTTDP(
    pois_df=cultural_pois,
    travel_time_matrix=cultural_travel_matrix,
    num_days=2,
    max_time_per_day=8,
    time_limit=300  # 5 minutes time limit
)

# Run the solver and measure time
print("Running MIP Solver for Premium Cultural Tour...")
print("This may take a few minutes to find the optimal solution...\n")
start_time = time.time()
itinerary, score, status = mip_cultural.solve(verbose=True)
execution_time = time.time() - start_time

print(f"\n{'='*70}")
print(f"✓ Optimization completed in {execution_time:.2f} seconds")
print(f"✓ Status: {status}")
print(f"✓ Total interest score: {score:.0f} (OPTIMAL)")
print(f"{'='*70}")

In [ ]:
# Display the optimal itinerary
print("\n" + "="*70)
print("OPTIMAL CULTURAL TOUR ITINERARY")
print("="*70 + "\n")

for day_idx, day_pois in enumerate(itinerary, 1):
    print(f"\n📅 Day {day_idx}:")
    print("-" * 70)
    
    day_time = 0
    day_score = 0
    
    for visit_idx, poi_idx in enumerate(day_pois, 1):
        poi = cultural_pois.iloc[poi_idx]
        
        # Calculate travel time from previous POI
        if visit_idx > 1:
            prev_poi_idx = day_pois[visit_idx - 2]
            travel_time = cultural_travel_matrix[prev_poi_idx, poi_idx]
            day_time += travel_time
        
        # Add visit time
        day_time += poi['visit_duration']
        day_score += poi['interest_score']
        
        print(f"  {visit_idx}. {poi['name']}")
        print(f"     Rating: {'⭐' * int(poi['rating'])} ({poi['rating']:.1f})")
        print(f"     Interest Score: {poi['interest_score']:.0f} | Visit Duration: {poi['visit_duration']:.1f}h")
    
    print(f"\n  📊 Day Summary: {len(day_pois)} attractions | {day_time:.1f}h total | Score: {day_score:.0f}")

print(f"\n{'='*70}")
print(f"Total POIs: {sum(len(day) for day in itinerary)} | Total Score: {score:.0f} (OPTIMAL)")
print(f"{'='*70}")

---
## Use Case 2: Wildlife Safari Optimization (2 Days)

**Scenario**: Plan the perfect wildlife safari visiting national parks and wildlife reserves. We need the optimal route to maximize wildlife viewing opportunities.

**Requirements**:
- Top 20 wildlife POIs
- 2 days trip
- 8 hours per day
- Optimal scheduling for best experience

In [ ]:
# Filter and select wildlife POIs
wildlife_pois = pois_full[
    (pois_full['category'] == 'Wildlife') & 
    (pois_full['rating'] >= 3.5)
].nlargest(20, 'interest_score').reset_index(drop=True)

# Get corresponding travel time matrix
wildlife_indices = pois_full[
    (pois_full['category'] == 'Wildlife') & 
    (pois_full['rating'] >= 3.5)
].nlargest(20, 'interest_score').index.tolist()
wildlife_travel_matrix = travel_matrix_full[np.ix_(wildlife_indices, wildlife_indices)]

print(f"Selected top {len(wildlife_pois)} wildlife POIs")
print(f"\nTop attractions:")
print(wildlife_pois.head(10)[['name', 'rating', 'interest_score', 'visit_duration']])

In [ ]:
# Configure MIP solver for wildlife tour
mip_wildlife = MIPSolverTTDP(
    pois_df=wildlife_pois,
    travel_time_matrix=wildlife_travel_matrix,
    num_days=2,
    max_time_per_day=8,
    time_limit=300
)

# Run the solver
print("Running MIP Solver for Wildlife Safari...\n")
start_time = time.time()
itinerary_w, score_w, status_w = mip_wildlife.solve(verbose=True)
execution_time_w = time.time() - start_time

print(f"\n{'='*70}")
print(f"✓ Optimization completed in {execution_time_w:.2f} seconds")
print(f"✓ Status: {status_w}")
print(f"✓ Total interest score: {score_w:.0f} (OPTIMAL)")
print(f"{'='*70}")

In [ ]:
# Display the optimal itinerary
print("\n" + "="*70)
print("OPTIMAL WILDLIFE SAFARI ITINERARY")
print("="*70 + "\n")

for day_idx, day_pois in enumerate(itinerary_w, 1):
    print(f"\n📅 Day {day_idx}:")
    print("-" * 70)
    
    day_time = 0
    day_score = 0
    
    for visit_idx, poi_idx in enumerate(day_pois, 1):
        poi = wildlife_pois.iloc[poi_idx]
        
        if visit_idx > 1:
            prev_poi_idx = day_pois[visit_idx - 2]
            travel_time = wildlife_travel_matrix[prev_poi_idx, poi_idx]
            day_time += travel_time
        
        day_time += poi['visit_duration']
        day_score += poi['interest_score']
        
        print(f"  {visit_idx}. 🦁 {poi['name']}")
        print(f"     Rating: {'⭐' * int(poi['rating'])} ({poi['rating']:.1f})")
        print(f"     Interest Score: {poi['interest_score']:.0f} | Visit Duration: {poi['visit_duration']:.1f}h")
    
    print(f"\n  📊 Day Summary: {len(day_pois)} locations | {day_time:.1f}h total | Score: {day_score:.0f}")

print(f"\n{'='*70}")
print(f"Total POIs: {sum(len(day) for day in itinerary_w)} | Total Score: {score_w:.0f} (OPTIMAL)")
print(f"{'='*70}")

---
## Use Case 3: Waterfall Tour with Tight Schedule (1 Day)

**Scenario**: A single day to visit as many beautiful waterfalls as possible. Need the optimal route to maximize the number of waterfalls visited.

**Requirements**:
- Top 15 waterfalls
- 1 day only
- 8 hours available
- Maximize coverage

In [ ]:
# Filter and select waterfall POIs
waterfall_pois = pois_full[
    (pois_full['category'] == 'Waterfalls') & 
    (pois_full['rating'] >= 4.0)
].nlargest(15, 'interest_score').reset_index(drop=True)

# Get corresponding travel time matrix
waterfall_indices = pois_full[
    (pois_full['category'] == 'Waterfalls') & 
    (pois_full['rating'] >= 4.0)
].nlargest(15, 'interest_score').index.tolist()
waterfall_travel_matrix = travel_matrix_full[np.ix_(waterfall_indices, waterfall_indices)]

print(f"Selected top {len(waterfall_pois)} waterfall POIs")
print(f"\nTop waterfalls:")
print(waterfall_pois[['name', 'rating', 'interest_score', 'visit_duration']])

In [ ]:
# Configure MIP solver for waterfall tour
mip_waterfall = MIPSolverTTDP(
    pois_df=waterfall_pois,
    travel_time_matrix=waterfall_travel_matrix,
    num_days=1,
    max_time_per_day=8,
    time_limit=180  # 3 minutes should be enough for 15 POIs
)

# Run the solver
print("Running MIP Solver for Waterfall Tour...\n")
start_time = time.time()
itinerary_wf, score_wf, status_wf = mip_waterfall.solve(verbose=True)
execution_time_wf = time.time() - start_time

print(f"\n{'='*70}")
print(f"✓ Optimization completed in {execution_time_wf:.2f} seconds")
print(f"✓ Status: {status_wf}")
print(f"✓ Total interest score: {score_wf:.0f} (OPTIMAL)")
print(f"{'='*70}")

In [ ]:
# Display the optimal itinerary
print("\n" + "="*70)
print("OPTIMAL WATERFALL TOUR ITINERARY")
print("="*70 + "\n")

for day_idx, day_pois in enumerate(itinerary_wf, 1):
    print(f"\n📅 Day {day_idx}:")
    print("-" * 70)
    
    day_time = 0
    day_score = 0
    
    for visit_idx, poi_idx in enumerate(day_pois, 1):
        poi = waterfall_pois.iloc[poi_idx]
        
        if visit_idx > 1:
            prev_poi_idx = day_pois[visit_idx - 2]
            travel_time = waterfall_travel_matrix[prev_poi_idx, poi_idx]
            day_time += travel_time
        
        day_time += poi['visit_duration']
        day_score += poi['interest_score']
        
        print(f"  {visit_idx}. 💧 {poi['name']}")
        print(f"     Rating: {'⭐' * int(poi['rating'])} ({poi['rating']:.1f})")
        print(f"     Interest Score: {poi['interest_score']:.0f} | Visit Duration: {poi['visit_duration']:.1f}h")
    
    print(f"\n  📊 Day Summary: {len(day_pois)} waterfalls | {day_time:.1f}h total | Score: {day_score:.0f}")

print(f"\n{'='*70}")
print(f"Total POIs: {sum(len(day) for day in itinerary_wf)} | Total Score: {score_wf:.0f} (OPTIMAL)")
print(f"{'='*70}")

---
## Performance Analysis: MIP vs Genetic Algorithm

Let's compare the MIP solver's optimal solutions with what the Genetic Algorithm would produce for the same problems.

In [ ]:
# Import GA for comparison
from genetic_algorithm import GeneticAlgorithmTTDP

# Run GA on the same cultural tour problem
print("Running GA for comparison on Cultural Tour...\n")
ga_cultural = GeneticAlgorithmTTDP(
    pois_df=cultural_pois,
    travel_time_matrix=cultural_travel_matrix,
    num_days=2,
    max_time_per_day=8,
    population_size=200,
    generations=500
)

start_ga = time.time()
_, _, ga_score = ga_cultural.evolve(verbose=False)
ga_time = time.time() - start_ga

print(f"GA Score: {ga_score:.0f} in {ga_time:.2f}s")
print(f"MIP Score: {score:.0f} in {execution_time:.2f}s")
print(f"\nGA Quality: {(ga_score/score)*100:.2f}% of optimal")
print(f"MIP is {ga_time/execution_time:.2f}x slower but guarantees optimality")

In [ ]:
# Create visual comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Score comparison
methods = ['MIP\n(Optimal)', 'Genetic\nAlgorithm']
scores = [score, ga_score]
colors = ['#2ecc71', '#3498db']

bars1 = ax1.bar(methods, scores, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax1.set_ylabel('Total Interest Score', fontsize=12)
ax1.set_title('Solution Quality Comparison\n(Cultural Tour - 2 Days)', fontsize=13, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.0f}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

# Time comparison (log scale for better visualization)
times = [execution_time, ga_time]
bars2 = ax2.bar(methods, times, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax2.set_ylabel('Execution Time (seconds)', fontsize=12)
ax2.set_title('Computation Speed Comparison\n(Lower is Better)', fontsize=13, fontweight='bold')
ax2.set_yscale('log')
ax2.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}s',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('../results/mip_vs_ga_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Comparison plot saved to results/mip_vs_ga_comparison.png")

---
## Summary: MIP Performance Overview

In [ ]:
# Create summary table
summary_data = {
    'Use Case': ['Cultural Tour', 'Wildlife Safari', 'Waterfall Tour'],
    'Days': [2, 2, 1],
    'POIs Available': [len(cultural_pois), len(wildlife_pois), len(waterfall_pois)],
    'POIs Visited': [
        sum(len(day) for day in itinerary),
        sum(len(day) for day in itinerary_w),
        sum(len(day) for day in itinerary_wf)
    ],
    'Optimal Score': [score, score_w, score_wf],
    'Solve Time (s)': [execution_time, execution_time_w, execution_time_wf],
    'Status': [status, status_w, status_wf]
}

summary_df = pd.DataFrame(summary_data)

print("\n" + "="*70)
print("MIP SOLVER PERFORMANCE SUMMARY")
print("="*70 + "\n")
print(summary_df.to_string(index=False))
print("\n" + "="*70)

# Save to CSV
summary_df.to_csv('../results/mip_demo_summary.csv', index=False)
print("\n✓ Summary saved to results/mip_demo_summary.csv")

---
## Conclusion

This demo showcased the **Mixed-Integer Programming** approach for the Tourist Trip Design Problem with three realistic use cases:

1. **Premium Cultural Tour**: Optimal 2-day cultural heritage experience
2. **Wildlife Safari**: Optimal wildlife viewing itinerary
3. **Waterfall Tour**: Maximum waterfall coverage in one day

### Key Takeaways:

✅ **Optimality**: MIP provides mathematically proven optimal solutions

✅ **Reliability**: Always finds the best possible itinerary given constraints

✅ **Transparency**: Solution status clearly indicates if optimum was reached

⚠️ **Trade-off**: Slower than heuristic methods (seconds vs. minutes)

### When to Use MIP:

✔️ When you need the **absolute best** solution

✔️ For smaller problem instances (< 50 POIs)

✔️ When you can afford longer computation times

✔️ When solution quality must be provable/certified

✔️ For high-value planning (premium tours, limited-time trips)

### When to Use Genetic Algorithm Instead:

✔️ Large number of POIs (100+)

✔️ Need results immediately

✔️ Near-optimal is good enough (99%+ quality)

✔️ Interactive/exploratory planning

---

**Recommendation**: Use MIP for final itinerary optimization when quality matters most, and GA for quick exploration and large-scale planning!